## 1. Show Samples 

In [28]:
import os
import cv2
from ipywidgets import interact
import matplotlib.pyplot as plt
%matplotlib inline

In [29]:
source_image_dir = "../resources/export_frame/lidar01/"
image_files = sorted([fn for fn in os.listdir(source_image_dir) if fn.endswith("jpg")])

In [31]:
@interact(index=(0, len(image_files)-1))
def show_sample(index=0):
    image_file = image_files[index]
    image_path = os.path.join(source_image_dir, image_file)
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    plt.figure(figsize=(16,16))
    plt.imshow(image)
    plt.axis('off')
    plt.show()

interactive(children=(IntSlider(value=0, description='index', max=654), Output()), _dom_classes=('widget-inter…

## 2. Save Specific frame

In [42]:
save_image_dir = "../resources/save_test/lidar01/"

if not os.path.exists(save_image_dir):
    os.makedirs(save_image_dir)

In [41]:
# some handy functions to use along widgets
from IPython.display import display, Markdown, clear_output
# widget packages
import ipywidgets as widgets
    
button = widgets.Button(description='My Button')
out = widgets.Output()

@interact(index=(0, len(image_files)-1))
def show_sample(index=0):
    image_file = image_files[index]
    image_path = os.path.join(source_image_dir, image_file)
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    def on_button_clicked(_):
        # "linking function with output"
        with out:
            # what happens when we press the button
            clear_output()
            
    # linking button and function together using a button's method
    button.on_click(on_button_clicked)
    # displaying button and its output together
   
    
    plt.figure(figsize=(16,16))
    plt.imshow(image)
    plt.axis('off')
    plt.show()

widgets.VBox([button,out])

interactive(children=(IntSlider(value=0, description='index', max=654), Output()), _dom_classes=('widget-inter…

## 3. Save FP/FN Images

In [ ]:
def process_one_image(preds: torch.Tensor, targets: torch.Tensor, match_metric):
    tp_boxes = None
    fp_boxes = None
    fn_boxes = None
    iou = box_iou(targets[:, 1:], preds[:, :4])
    correct_class = targets[:, 0:1] == preds[:, 5]
    x = torch.where((iou >= match_metric) & (correct_class))
    if x[0].numel():
        matches = torch.cat((torch.stack(x, 1), iou[x[0], x[1]][:, None]), 1).cpu().numpy()
        if x[0].shape[0] > 1:
            matches = matches[matches[:, 2].argsort()[::-1]]
            matches = matches[np.unique(matches[:, 1], return_index=True)[1]]
            matches = matches[matches[:, 2].argsort()[::-1]]
            matches = matches[np.unique(matches[:, 0], return_index=True)[1]]
        tp_boxes = preds[matches[:, 1]]
        fp_idxs = [idx for idx in range(preds.shape[0]) if idx not in matches[:, 1]]
        fp_boxes = preds[fp_idxs]
        fn_idxs = [idx for idx in range(targets.shape[0]) if idx not in matches[:, 0]]
        fn_boxes = targets[fn_idxs]
    else:
        fn_boxes = targets
    return tp_boxes if tp_boxes is not None else torch.tensor([]), fp_boxes if fp_boxes is not None else torch.tensor([]),\
        fn_boxes if fn_boxes is not None else torch.tensor([])

# eval_one_image(im[si], pred, ((labels[:, 0:1], xywh2xyxy(labels[:, 1:5]))), path, names, save_dir)

def eval_one_image(img, preds, targets, path, names, save_dir, conf_metric=0.3, match_metrics=0.5):
    """
    img: torch.Tensor
    preds: torch.Tensor[x1, y1, x2, y2, conf, class_id]
    targets: torch.Tensor[class_id, x1, y1, x2, y2]
    """
    (save_dir / 'tps').mkdir(parents=True, exist_ok=True)
    (save_dir / 'fps').mkdir(parents=True, exist_ok=True)
    (save_dir / 'fns').mkdir(parents=True, exist_ok=True)
    (save_dir / 'combined').mkdir(parents=True, exist_ok=True)
    preds = preds[preds[:, 4] > conf_metric]
    tps, fps, fns = process_one_image(preds, targets, match_metrics)
    pil_img = to_pil_image(img)
    pil_tp_img = to_pil_image(img)
    pil_fp_img = to_pil_image(img)
    pil_fn_img = to_pil_image(img)
    pil_draw = ImageDraw.Draw(pil_img)
    pil_tp_draw = ImageDraw.Draw(pil_tp_img)
    pil_fp_draw = ImageDraw.Draw(pil_fp_img)
    pil_fn_draw = ImageDraw.Draw(pil_fn_img)
    if tps.numel():
        for tp in tps:
            x1, y1, x2, y2, conf, class_id = tp
            pil_tp_draw.rectangle((x1, y1, x2, y2), outline='red', width=3)
            pil_tp_draw.text((x1, y1 - 22), f'{names[int(class_id)]} {conf:.2f}', fill='red', font=ImageFont.truetype('arial.ttf', size=20))
            pil_draw.rectangle((x1, y1, x2, y2), outline='blue', width=2)
            pil_draw.text((x1, y1 - 22), f'{names[int(class_id)]} {conf:.2f}', fill='blue', font=ImageFont.truetype('arial.ttf', size=13))
    if fps.numel():
        for fp in fps:
            x1, y1, x2, y2, conf, class_id = fp
            pil_fp_draw.rectangle((x1, y1, x2, y2), outline='red', width=3)
            pil_fp_draw.text((x1, y1 - 22), f'{names[int(class_id)]} {conf:.2f}', fill='red', font=ImageFont.truetype('arial.ttf', size=20))
            pil_draw.rectangle((x1, y1, x2, y2), outline='yellow', width=2)
            pil_draw.text((x1, y1 - 22), f'{names[int(class_id)]} {conf:.2f}', fill='yellow', font=ImageFont.truetype('arial.ttf', size=13))
    if fns.numel():
        for fn in fns:
            class_id, x1, y1, x2, y2 = fn
            pil_fn_draw.rectangle((x1, y1, x2, y2), outline='red', width=3)
            pil_fn_draw.text((x1, y1 - 22), f'{names[int(class_id)]}', fill='red', font=ImageFont.truetype('arial.ttf', size=20))
            pil_draw.rectangle((x1, y1, x2, y2), outline='red', width=2)
            pil_draw.text((x1, y1 - 22), f'{names[int(class_id)]}', fill='red', font=ImageFont.truetype('arial.ttf', size=13))
    pil_tp_img.save(save_dir / 'tps' / f'{path.stem}.jpg')
    pil_fp_img.save(save_dir / 'fps' / f'{path.stem}.jpg')
    pil_fn_img.save(save_dir / 'fns' / f'{path.stem}.jpg')
    pil_img.save(save_dir / 'combined' / f'{path.stem}.jpg')

## 3. Reference

In [15]:
# https://medium.com/@technologger/how-to-interact-with-jupyter-33a98686f24e

In [9]:
# some handy functions to use along widgets
from IPython.display import display, Markdown, clear_output
# widget packages
import ipywidgets as widgets
# defining some widgets
text = widgets.Text(
       value='My Text',
       description='Title', )
calendar = widgets.DatePicker(
           description='Select Date')
slider = widgets.FloatSlider(
         value=1,
         min=0,
         max=10.0,
         step=0.1,)
menu = widgets.Dropdown(
       options=['red', 'blue', 'green'],
       value='red',
       description='Color:')
checkbox = widgets.Checkbox(
           description='Check to invert',)

In [11]:
menu

Dropdown(description='Color:', options=('red', 'blue', 'green'), value='red')

In [13]:
box = widgets.VBox([text, slider, menu, calendar, checkbox ])
box

In [14]:
widgets.HBox([box, box])

In [16]:
# defining a list with the contents of our windows
children = [box, box]
# initializing a tab
tab = widgets.Tab()
# setting the tab windows 
tab.children = children
# changing the title of the first and second window
tab.set_title(0, 'box')
tab.set_title(1, 'copy of box')
tab

In [17]:
menu.options=('red', 'blue', 'green', 'black')
menu

Dropdown(description='Color:', options=('red', 'blue', 'green', 'black'), value='red')

In [19]:
# current value of text, checkbox and calendar 
print('Widget text has value {}, of {}'.format(text.value, type(text.value)))
print('Widget checkbox has value {}, of {}'.format(checkbox.value, type(checkbox.value)))
print('Widget calendar has value {}, of {}'.format(calendar.value, type(calendar.value)))

Widget text has value My Text ㅁㄴㅇㅁㄴㅇ, of <class 'str'>
Widget checkbox has value False, of <class 'bool'>
Widget calendar has value None, of <class 'NoneType'>


In [20]:
button = widgets.Button(description='My Button')

In [22]:
button = widgets.Button(description='My Button')
out = widgets.Output()
def on_button_clicked(_):
    # "linking function with output"
    with out:
        # what happens when we press the button
        clear_output()
        print('Something happens!')
# linking button and function together using a button's method
button.on_click(on_button_clicked)
# displaying button and its output together
widgets.VBox([button,out])


In [23]:
# some options
a, b = 1, 5
# selecting global variables without underscores
global_variables = list(globals().keys())
funcs = []
for i in global_variables:
    if '_' not in i:
        funcs.append(i)
 
# creating menu with them
global_vars = widgets.Dropdown(
    options=funcs)
# button, output, function and linkage
butt = widgets.Button(description='Print Variable')
outt = widgets.Output()
def on_butt_clicked(b):
    with outt:
        clear_output()
        print(type(globals()[global_vars.value]))
        print(globals()[global_vars.value])
        
butt.on_click(on_butt_clicked)
# display
widgets.VBox([global_vars,butt,outt])